# Process NO2 data from Cloud Raster Format file

In [4]:
from arcgis.gis import GIS
from arcgis.raster import RasterCollection, Raster
from arcgis import raster
import arcpy
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
gis= GIS()

In [5]:
base_path = r'Y:\Documents\GIS_data\Imagery\sentinel-5p\ny-2019-2020\ny_crf'
crf_path = os.path.join(base_path, 'ny_19_20.crf')
crf_path

'Y:\\Documents\\GIS_data\\Imagery\\sentinel-5p\\ny-2019-2020\\ny_crf\\ny_19_20.crf'

In [3]:
# download data
# nc_item = gis.content.get('b713f5b7fbd14c7597d9797b27b14835')
# nc_item.download(save_path=base_path)

<Item title:"New York NO2 levels April 2019, April 2020 as CRF" type:Image Collection owner:api_data_owner>

#### degbug lines
```python
from arcgis.raster import RasterCollection, Raster
from arcgis import raster
import arcpy
crf_path = r'Y:\Documents\GIS_data\Imagery\sentinel-5p\ny-2019-2020\ny_crf\ny_19_20.crf'
all_rc = RasterCollection(crf_path)
all_2019 = all_rc.filter_by_calendar_range('YEAR',2019,2019)
no2_19 = all_2019.filter_by_attribute(field_name='Variable', 
                                          operator='CONTAINS', 
                                          field_values='nitrogendioxide')
r1 = no2_19[0]['Raster']
```

## Read into a `RasterCollection` object and extract 1st `Raster`

In [6]:
all_rc = RasterCollection(crf_path)
all_2019 = all_rc.filter_by_calendar_range('YEAR',2019,2019)
no2_19 = all_2019.filter_by_attribute(field_name='Variable', 
                                          operator='CONTAINS', 
                                          field_values='nitrogendioxide')
r1 = no2_19[0]['Raster']
type(r1)

arcgis.raster._layer.Raster

In [8]:
# get arcpy raster
arcpy_r1 = r1._engine_obj._raster
type(arcpy_r1)

arcpy.sa.Raster.Raster

## Bug 1 - map algebra difference (Greater Than Equal)

In [10]:
r1_pos_mask = r1 >= 0
(r1_pos_mask.minimum, r1_pos_mask.maximum) # should be 0, 1

(-7.956242916407064e-05, 0.0158010832965374)

In [11]:
arcpy_r1_pos_mask = arcpy_r1 >=0
(arcpy_r1_pos_mask.minimum, arcpy_r1_pos_mask.maximum)

(0.0, 1.0)

## Bug 2 - logical raster function difference

In [12]:
r1_con = raster.functions.con([(r1>=0), r1, 0])
print(r1_con.minimum, r1_con.maximum) # throws error or returns wrong answer 

Failed to apply Raster Function: '{"rasterFunction": "Local", "rasterFunctionArguments": {"Rasters": [{"rasterFunction": "Local", "rasterFunctionArguments": {"Rasters": [0, "C:\\Users\\atma6951\\AppData\\Local\\Temp\\xcbe99442_6bdf_4adc_993b_1bf9e13eb9c6y0.afr"], "Operation": 29, "ExtentType": 0, "CellsizeType": 0}, "variableName": "Rasters"}, "C:\\Users\\atma6951\\AppData\\Local\\Temp\\xcbe99442_6bdf_4adc_993b_1bf9e13eb9c6y0.afr", 0], "Operation": 76, "ExtentType": 0, "CellsizeType": 0}, "variableName": "Rasters"}' (Unspecified error )
-7.956242916407064e-05 0.0158010832965374


In [13]:
arcpy_r1_con = arcpy.ia.Con(arcpy_r1_pos_mask, arcpy_r1)
print(arcpy_r1_con.minimum, arcpy_r1_con.maximum) # returns only positive result which is correct

9.787409077643972e-11 0.0004633407515939325
